In [7]:
import os

def write_grammar_to_file(grammar,file_path='grammar.txt'):
    grammar_file_content = ""
    for lhs in grammar:
        for rhs in grammar[lhs]:
            grammar_file_content += f"{lhs} = {' '.join(rhs)}\n"

    # Write the grammar to a file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(grammar_file_content.strip())

def read_grammar(file_path):
    grammar = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:
                lhs, rhs = line.split('=')
                lhs = lhs.strip()
                rhs = rhs.strip().split()
                if lhs not in grammar:
                    grammar[lhs] = []
                grammar[lhs].append(rhs)
    return grammar

def compute_first_sets(grammar):
    first = {non_terminal: set() for non_terminal in grammar}

    def first_of(symbol):
        if symbol not in grammar:  # Terminal symbol
            return {symbol}
        if not first[symbol]:  # Non-terminal, not yet computed
            for production in grammar[symbol]:
                for sym in production:
                    sym_first = first_of(sym)
                    first[symbol].update(sym_first)
                    if 'ɛ' not in sym_first:
                        break
                else:
                    first[symbol].add('ɛ')
        return first[symbol]

    for non_terminal in grammar:
        first_of(non_terminal)
    return first

def compute_follow_sets(grammar, start_symbol, first):
    follow = {non_terminal: set() for non_terminal in grammar}
    follow[start_symbol].add('$')

    def follow_of(non_terminal):
        for lhs in grammar:
            for production in grammar[lhs]:
                for i, sym in enumerate(production):
                    if sym == non_terminal:
                        if i + 1 < len(production):
                            next_symbol = production[i + 1]
                            if next_symbol in grammar:  # Non-terminal
                                follow[non_terminal].update(first[next_symbol] - {'ɛ'})
                                if 'ɛ' in first[next_symbol]:
                                    follow[non_terminal].update(follow_of(lhs))
                            else:  # Terminal
                                follow[non_terminal].add(next_symbol)
                        else:
                            if lhs != non_terminal:
                                follow[non_terminal].update(follow_of(lhs))
        return follow[non_terminal]

    for non_terminal in grammar:
        follow_of(non_terminal)
    return follow


# Ensure the file path is correct
file_path = 'grammar.txt'  # Update this path if necessary




# Read the grammar from file
grammar = read_grammar(file_path)
start_symbol = next(iter(grammar))
first = compute_first_sets(grammar)
follow = compute_follow_sets(grammar, start_symbol, first)

# Print FIRST and FOLLOW sets
print("FIRST sets:")
for non_terminal in first:
    print(f"{non_terminal} = {first[non_terminal]}")

print("\nFOLLOW sets:")
for non_terminal in follow:
    print(f"{non_terminal} = {follow[non_terminal]}")


FIRST sets:
S = {'ɛ', 'b', 'a'}
X = {'ɛ', 'b', 'a'}

FOLLOW sets:
S = {'$'}
X = {'a'}
